In [14]:
#Seq2seq learning. In this case for audio samples. But can be generalized to text with proper encodings. I used the 
#image for these cases, but what's in a name ? :D



import sys
import os
import cv2
import numpy as np
import tensorflow
import tensorflow as tf
import keras
import keras, keras.layers as L
sys.path.append("..")
from skimage.color import rgb2gray
import keras
from keras.models import load_model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import os
import pickle
import numpy as np

import os
import cv2

import numpy as np
import pandas as pd
from scipy import signal
from scipy.io import wavfile
import matplotlib.pyplot as plt

In [15]:
import numpy as np
def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                   if os.path.isdir(os.path.join(data_directory, d))]
    labels = []
    sounds = []
    for d in directories:
        label_directory = os.path.join(data_directory, d)
        file_names = [os.path.join(label_directory, f) 
                      for f in os.listdir(label_directory) 
                      if f.endswith(".wav")]
        for f in file_names:
            fs, dat = (wavfile.read(f))
            sounds.append((dat))
            labels.append(d)
    return sounds, labels

In [16]:
X,y = load_data('speech_commands_v0.01/')

/Users/bedabratachoudhury/anaconda2/lib/python2.7/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [23]:
xtrain=np.array(([cv2.resize(image, (1, 16000)) for image in X]))


(64727, 16000, 1)

In [33]:
img_shape = xtrain[0].shape
def build_pca_autoencoder(img_shape,code_size=32):
    """
    Here we define a simple linear autoencoder as described above.
    We also flatten and un-flatten data to be compatible with image shapes
    """
    
    encoder = keras.models.Sequential()
    encoder.add(L.InputLayer(img_shape))
    encoder.add(L.Conv1D(filters=100, kernel_size=4, padding='same', activation='relu'))
    encoder.add(L.LSTM(100,return_sequences=True))
    encoder.add(L.Flatten())                  #flatten image to vector
    encoder.add(L.Dense(code_size))           #actual encoder
    
    
    decoder = keras.models.Sequential()
    decoder.add(L.InputLayer((code_size,)))
    decoder.add(L.Dense(np.prod(img_shape)))  #actual decoder, height*width*3 units
    decoder.add(L.Reshape(img_shape))#un-flatten
    return encoder,decoder



In [34]:
encoder,decoder = build_pca_autoencoder(img_shape,code_size=32)

inp = L.Input(img_shape)
code = encoder(inp)
reconstruction = decoder(code)

autoencoder = keras.models.Model(inp,reconstruction)
autoencoder.compile('adamax','mse')

In [ ]:
autoencoder.fit(x=xtrain,y=xtrain,epochs=32)

Epoch 1/32
